In [ ]:
########## Raw Data 다운 ##########

import requests
import pandas as pd
import time
import json
from datetime import datetime

symbol = "BTCUSDT"
interval = "1h"
limit = 1000

# 2017-08-17 00:00 UTC (Binance BTC 최초 데이터 시점)
start_timestamp = int(pd.Timestamp("2017-08-17 00:00:00", tz='UTC').timestamp() * 1000)

checkpoint_file = "checkpoint.json"
output_file = "btc_raw.csv"

# ------------------------------------------
# Load checkpoint
# ------------------------------------------
def load_checkpoint():
    try:
        with open(checkpoint_file, "r") as f:
            return json.load(f)
    except:
        return {"last_time": start_timestamp}

# ------------------------------------------
# Save checkpoint
# ------------------------------------------
def save_checkpoint(ts):
    with open(checkpoint_file, "w") as f:
        json.dump({"last_time": ts}, f)

# ------------------------------------------
# Request to Binance API (with retry logic)
# ------------------------------------------
def fetch_klines(start_time):
    url = "https://api.binance.com/api/v3/klines"
    params = {
        "symbol": symbol,
        "interval": interval,
        "startTime": start_time,
        "limit": limit
    }

    while True:
        try:
            r = requests.get(url, params=params, timeout=10)

            if r.status_code == 200:
                return r.json()

            # Binance rate limit
            if r.status_code in [418, 429]:
                print("Rate limit hit → waiting 60 seconds...")
                time.sleep(60)

            # Temporary block or legal restriction
            elif r.status_code in [403, 451]:
                print(f"Server blocked ({r.status_code}) → waiting 30 seconds...")
                time.sleep(30)

            else:
                print(f"Unexpected status {r.status_code}, retrying in 15s...")
                time.sleep(15)

        except Exception as e:
            print("Connection error:", e, "retry in 10s...")
            time.sleep(10)

# ------------------------------------------
# Main loop
# ------------------------------------------
if __name__ == "__main__":

    cp = load_checkpoint()
    current_start = cp["last_time"]

    print("Starting download from:", pd.to_datetime(current_start, unit='ms'))

    end_time = int(time.time() * 1000)
    all_data = []

    while current_start < end_time:
        klines = fetch_klines(current_start)

        if not klines:
            print("No more data returned.")
            break

        all_data.extend(klines)
        last_close_time = klines[-1][6]

        # Save checkpoint
        save_checkpoint(last_close_time)
        current_start = last_close_time

        print(
            f"Fetched {len(klines)} rows, "
            f"next start = {pd.to_datetime(current_start, unit='ms')}"
        )

        time.sleep(0.4)  # Prevent rate limit

    print("Download complete. Converting to DataFrame...")

# ------------------------------------------
# Convert result to DataFrame
# ------------------------------------------
cols = [
    "OpenTime", "Open", "High", "Low", "Close", "Volume",
    "CloseTime", "QuoteAssetVolume", "NumberOfTrades",
    "TakerBuyBase", "TakerBuyQuote", "Ignore"
]

df = pd.DataFrame(all_data, columns=cols)

# Convert timestamps
df["OpenTime"] = pd.to_datetime(df["OpenTime"], unit="ms")
df["CloseTime"] = pd.to_datetime(df["CloseTime"], unit="ms")

# Remove duplicates
df = df.drop_duplicates(subset="OpenTime")

# Sort
df = df.sort_values("OpenTime")

# ⚠ 여기서 필요한 5개 컬럼만 선택
df = df[["OpenTime", "Open", "High", "Low", "Close"]]

# Save CSV
df.to_csv(output_file, index=False)
print("Saved:", output_file)

In [ ]:
########## 전처리 ##########

import pandas as pd
import numpy as np

# =========================================================
# 1. Raw 데이터 불러오기
# =========================================================
df = pd.read_csv("btc_raw.csv")

# =========================================================
# 2. 시간 인덱스 정리 + 결측 29개 자동 보간
# =========================================================

# Binance 다운로드 스크립트에서는 "OpenTime" 컬럼이 날짜
df.rename(columns={"OpenTime": "Date"}, inplace=True)

df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values("Date").reset_index(drop=True)

# 1시간 단위 timestamp 강제 생성
df = df.set_index("Date").asfreq("1h")

# OHLC는 모두 직전값으로 채움 (forward fill)
df["Open"] = df["Open"].ffill()
df["High"] = df["High"].ffill()
df["Low"]  = df["Low"].ffill()
df["Close"] = df["Close"].ffill()

# index 복구
df = df.reset_index()

# =========================================================
# 3. Feature Engineering
# =========================================================

# 3.1 주가 기반 피처
df["log_return"] = np.log(df["Close"] / df["Close"].shift(1))
df["abs_return"] = df["log_return"].abs()
df["high_low_range"] = (df["High"] - df["Low"]) / df["Open"]

df["return_5"] = (df["Close"] - df["Close"].shift(4)) / df["Close"].shift(4)
df["return_24"] = (df["Close"] - df["Close"].shift(23)) / df["Close"].shift(23)

df["close_open"] = (df["Close"] - df["Open"]) / df["Open"]

# 3.2 봉 기반 피처
df["real_body"] = (df["Close"] - df["Open"]).abs()
df["upper_shadow"] = df["High"] - df[["Open", "Close"]].max(axis=1)
df["lower_shadow"] = df[["Open", "Close"]].min(axis=1) - df["Low"]
df["range"] = df["High"] - df["Low"]

epsilon = 1e-9
safe_range = df["range"] + epsilon

df["body_ratio"]  = df["real_body"]   / safe_range
df["upper_ratio"] = df["upper_shadow"] / safe_range
df["lower_ratio"] = df["lower_shadow"] / safe_range

# 3.3 변동성 기반 피처
df["rolling_vol_24"] = df["log_return"].rolling(24).std()
df["rolling_vol_168"] = df["log_return"].rolling(168).std()

df["parkinson_vol"] = np.sqrt(
    (1 / (4 * np.log(2))) * (np.log(df["High"] / df["Low"]) ** 2)
)

# 3.4 추세 기반 피처
df["SMA_5"] = df["Close"].rolling(5).mean()
df["SMA_10"] = df["Close"].rolling(10).mean()
df["SMA_20"] = df["Close"].rolling(20).mean()

df["EMA_12"] = df["Close"].ewm(span=12, adjust=False).mean()
df["EMA_26"] = df["Close"].ewm(span=26, adjust=False).mean()

df["MACD"] = df["EMA_12"] - df["EMA_26"]

# 3.5 모멘텀 기반 피처 (RSI)
delta = df["Close"].diff()
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)

avg_gain = gain.rolling(14).mean()
avg_loss = loss.rolling(14).mean()

RS = avg_gain / avg_loss
df["RSI_14"] = 100 - (100 / (1 + RS))

# 3.6 기술적 변동성 피처
df["TR"] = np.maximum.reduce([
    df["High"] - df["Low"],
    (df["High"] - df["Close"].shift(1)).abs(),
    (df["Low"] - df["Close"].shift(1)).abs()
])

df["ATR_14"] = df["TR"].rolling(14).mean()

# 3.7 고차모멘트 피처
df["skewness_24"] = df["log_return"].rolling(24).skew()
df["kurtosis_24"] = df["log_return"].rolling(24).kurt()

# =========================================================
# 4. Up/Down 라벨 생성
# =========================================================
df["target"] = (df["Close"].shift(-1) > df["Close"]).astype(int)

df = df.iloc[:-1]  # 마지막 target 없음

# =========================================================
# 5. 결측치 제거 및 기간 제한
# =========================================================
df = df.dropna()
df = df[df["Date"] >= "2020-01-01"]

# =========================================================
# 6. 저장
# =========================================================
df.to_csv("btc_processed.csv", index=False)

print("btc_processed.csv 생성 완료!")


In [ ]:
########## EDA ##########

# ============================================================
# 0. 라이브러리 설치
# ============================================================
!pip install ydata-profiling
!pip install seaborn
!pip install xgboost

# ============================================================
# 1. 라이브러리 불러오기
# ============================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

plt.style.use("seaborn-v0_8")
sns.set_palette("deep")

# ============================================================
# 2. 데이터 로드 및 기본 전처리
# ============================================================
df = pd.read_csv("btc_processed.csv")

# Date 컬럼 datetime 변환
if "Date" in df.columns:
    df["Date"] = pd.to_datetime(df["Date"])

print("\n===== 데이터 기본 정보 =====")
print(df.info())
print(df.head())

# ============================================================
# 3. Target 비율
# ============================================================
# Target 비율 시각화
plt.figure(figsize=(4,4))
df["target"].value_counts(normalize=True).plot(kind='bar')
plt.title("Target Class Distribution")
plt.xticks([0,1], ["Down(0)", "Up(1)"], rotation=0)
plt.show()

# ============================================================
# 4. Profiling Report 생성
# ============================================================
profile = ProfileReport(df, title="BTC Profiling Report")
profile.to_file("btc_profiling_report.html")

# ============================================================
# 5. 딥러닝용 EDA (OHLC 중심)
# ============================================================

# OHLC 시계열
plt.figure(figsize=(14,4))
plt.plot(df["Date"], df["Close"], linewidth=0.7)
plt.title("BTC Close Price Over Time")
plt.xlabel("Date"); plt.ylabel("Close Price")
plt.show()

# 변동성
df["volatility"] = df["High"] - df["Low"]
plt.figure(figsize=(14,4))
plt.plot(df["Date"], df["volatility"], color='orange')
plt.title("Volatility (High - Low) Over Time")
plt.xlabel("Date"); plt.ylabel("Volatility")
plt.show()

# 수익률(return)
df["return"] = df["Close"].pct_change()
plt.figure(figsize=(6,4))
sns.histplot(df["return"].dropna(), bins=100, kde=True)
plt.title("Return Distribution")
plt.show()

# Up/Down별 수익률 분포
plt.figure(figsize=(6,4))
sns.boxplot(x=df["target"], y=df["return"])
plt.title("Return by Target Class")
plt.xticks([0,1], ["Down","Up"])
plt.show()

# ============================================================
# 6. 머신러닝용 Feature EDA (OHLC, Date 제외)
# ============================================================
drop_cols = ["Open","High","Low","Close","Date","target","return","volatility"]
ml_features = df.drop(columns=[c for c in drop_cols if c in df.columns])

# 상관 분석
plt.figure(figsize=(12,8))
sns.heatmap(ml_features.corr(), cmap="coolwarm", center=0)
plt.title("Correlation Heatmap (ML Features)")
plt.show()

# Target과 각 feature 관계 확인
for col in ml_features.columns:
    plt.figure(figsize=(6,4))
    sns.boxplot(x=df["target"], y=df[col])
    plt.title(f"{col} vs Target")
    plt.xticks([0,1], ["Down","Up"])
    plt.show()

# ============================================================
# 7. Feature Importance (Baseline XGBoost)
# ============================================================
X = ml_features
y = df["target"]

# 시계열 특성 유지 → shuffle=False
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

xgb = XGBClassifier(
    max_depth=4,
    learning_rate=0.1,
    n_estimators=200,
    eval_metric="logloss"
)

xgb.fit(X_train, y_train)

importance = pd.Series(
    xgb.feature_importances_,
    index=X.columns
).sort_values(ascending=False)

plt.figure(figsize=(12,4))
importance.plot(kind="bar")
plt.title("XGBoost Feature Importance")
plt.show()

# ============================================================
# 8. Train vs Test 분포 비교 (시계열 drift 여부)
# ============================================================
train = df.iloc[:-5000]
train = df.iloc[:-5000]
test = df.iloc[-5000:]

plt.figure(figsize=(10,4))
sns.kdeplot(train["return"].dropna(), label="Train")
sns.kdeplot(test["return"].dropna(), label="Test")
plt.title("Train vs Test Return Distribution")
plt.legend()
plt.show()

print("\n🎉 전체 EDA 완료! 생성된 리포트 파일:")
print("- btc_profiling_report.html (ydata profiling)")


In [ ]:
########## 딥러닝 ##########

# ----------------------------------------------------------------------
# 0. 라이브러리 설치 (Colab 환경 기준)
# ----------------------------------------------------------------------

!pip install darts[u] autogluon.tabular shap captum

# ----------------------------------------------------------------------
# 1. 환경 설정 및 데이터 로딩
# ----------------------------------------------------------------------

# ----------------------------------------------------------------------
# 1-1. 주요 라이브러리 불러오기
# ----------------------------------------------------------------------

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    accuracy_score,
    f1_score,
    confusion_matrix,
    roc_curve,
    auc
)

import torch
torch.manual_seed(42)
np.random.seed(42)

# DARTS
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import BlockRNNModel, TCNModel, TransformerModel
from darts.metrics import mape

# AutoGluon
from autogluon.tabular import TabularPredictor

# SHAP
import shap

# ----------------------------------------------------------------------
# 1-2. Raw 데이터 불러오기
# ----------------------------------------------------------------------

DATA_PATH = "btc_processed.csv"
df = pd.read_csv(DATA_PATH)

# Date -> datetime
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values("Date").reset_index(drop=True)

print("데이터 컬럼:", df.columns.tolist())
print(df.head())

# ----------------------------------------------------------------------
# 2. DARTS 기반 딥러닝 예측 파이프라인
# ----------------------------------------------------------------------

# ----------------------------------------------------------------------
# 2-1. TimeSeries 생성 (Target=Close / Covariates=OHLC)
# ----------------------------------------------------------------------

# target: Close (univariate)
series_y = TimeSeries.from_dataframe(
    df,
    time_col="Date",
    value_cols=["Close"],
    freq=None
)

# features: OHLC (multivariate)
series_X = TimeSeries.from_dataframe(
    df,
    time_col="Date",
    value_cols=["Open", "High", "Low", "Close"],
    freq=None
)

print("series_y:", series_y)
print("series_X:", series_X)

# ----------------------------------------------------------------------
# 2-2. 시계열 스케일링 (입력·출력 개별 스케일러 적용)
# ----------------------------------------------------------------------

scaler_y = Scaler()
scaler_X = Scaler()

series_y_scaled = scaler_y.fit_transform(series_y)
series_X_scaled = scaler_X.fit_transform(series_X)

# ----------------------------------------------------------------------
# 2-3. 시계열 데이터 분할 (Train/Val/Test; 85%/15%)
# ----------------------------------------------------------------------

N = len(series_y_scaled)

N_train_val = int(N * 0.85)
N_train     = int(N_train_val * 0.8)
N_val       = N_train_val - N_train
N_test      = N - N_train_val

print("총 길이 N:", N)
print("train:", N_train, "val:", N_val, "test:", N_test)

# y split
train_y, valtest_y = series_y_scaled.split_before(N_train)
val_y,   test_y    = valtest_y.split_before(N_val)

# X split
train_X, valtest_X = series_X_scaled.split_before(N_train)
val_X,   test_X    = valtest_X.split_before(N_val)

print("len(train_y), len(val_y), len(test_y):",
      len(train_y), len(val_y), len(test_y))

# ----------------------------------------------------------------------
# 2-4. AutoDL 후보 모델 정의 (LSTM / GRU / TCN / Transformer)
# ----------------------------------------------------------------------

input_chunk_length = 168
output_chunk_length = 1

common_torch_kwargs = dict(
    input_chunk_length=input_chunk_length,
    output_chunk_length=output_chunk_length,
    n_epochs=3,
    batch_size=32,
    random_state=42,
    optimizer_kwargs={"lr": 1e-3},
    force_reset=True,
    save_checkpoints=True,
)

models = []

# 1) LSTM
models.append((
    "LSTM",
    BlockRNNModel(
        model="LSTM",
        **common_torch_kwargs,
        model_name="LSTM_btc_close"
    )
))

# 2) GRU
models.append((
    "GRU",
    BlockRNNModel(
        model="GRU",
        **common_torch_kwargs,
        model_name="GRU_btc_close"
    )
))

# 3) TCN
models.append((
    "TCN",
    TCNModel(
        **common_torch_kwargs,
        model_name="TCN_btc_close"
    )
))

# 4) Transformer
models.append((
    "Transformer",
    TransformerModel(
        **common_torch_kwargs,
        model_name="Transformer_btc_close"
    )
))

# ----------------------------------------------------------------------
# 2-5. 모델 학습 및 Validation MAPE 기반 Best Model 선택
# ----------------------------------------------------------------------

best_mape = np.inf
best_model = None
best_name  = None
val_results = {}

for name, model in models:
    print("\n==============================")
    print(f"▶ Training model: {name}")
    print("==============================")

    model.fit(
        series=train_y,
        past_covariates=train_X,
        val_series=val_y,
        val_past_covariates=val_X,
        verbose=True,
    )

    # Validation 예측
    pred_val_scaled = model.predict(
        n=len(val_y),
        past_covariates=series_X_scaled
    )

    # 역스케일링
    val_pred = scaler_y.inverse_transform(pred_val_scaled)
    val_true = scaler_y.inverse_transform(val_y)

    mape_val = mape(val_true, val_pred)
    val_results[name] = mape_val

    print(f"🔹 {name} Val MAPE: {mape_val:.4f}")

    if mape_val < best_mape:
        best_mape = mape_val
        best_model = model
        best_name  = name

print("\n✅ Best model on Validation (MAPE 기준)")
print("best_name :", best_name)
print("best_mape :", best_mape)

# ----------------------------------------------------------------------
# 2-6. Test Forecast 수행 및 RMSE/MAE 성능 평가
# ----------------------------------------------------------------------

horizon = len(test_y)

pred_test_scaled = best_model.predict(
    n=len(test_y),
    past_covariates=series_X_scaled,
)

# 역스케일링
pred_test = scaler_y.inverse_transform(pred_test_scaled)
test_true = scaler_y.inverse_transform(test_y)

y_true = test_true.values().flatten()
y_pred = pred_test.values().flatten()

rmse_value = np.sqrt(mean_squared_error(y_true, y_pred))
mae_value  = mean_absolute_error(y_true, y_pred)

print("\n📌 DARTS DL 회귀 성능")
print(f"RMSE: {rmse_value:.4f}")
print(f"MAE : {mae_value:.4f}")

# ----------------------------------------------------------------------
# 2-7. Close 예측 기반 Up/Down 방향성 분류 및 정확도 평가
# ----------------------------------------------------------------------

# true_diff / pred_diff 정의
true_diff = np.diff(y_true)
pred_diff = np.diff(y_pred)

y_true_cls = (true_diff > 0).astype(int)
y_pred_cls = (pred_diff > 0).astype(int)

acc_cls = accuracy_score(y_true_cls, y_pred_cls)
f1_cls  = f1_score(y_true_cls, y_pred_cls)
cm      = confusion_matrix(y_true_cls, y_pred_cls)

print("\n📌 DL Direction Classification (Up/Down)")
print(f"Accuracy: {acc_cls:.4f}")
print(f"F1      : {f1_cls:.4f}")

# Confusion Matrix
plt.figure(figsize=(5, 4))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    linecolor=None,
    linewidths=0,
    xticklabels=["Pred Down", "Pred Up"],
    yticklabels=["True Down", "True Up"]
)

plt.grid(False)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix (Up/Down Classification)")
plt.show()

# ROC curve (score로 pred_diff 사용)
fpr, tpr, thresholds = roc_curve(y_true_cls, pred_diff)
roc_auc = auc(fpr, tpr)

print(f"ROC AUC: {roc_auc:.4f}")

# ROC plot
plt.figure(figsize=(5, 5))
plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.3f}")
plt.plot([0, 1], [0, 1], linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("DL Direction Classification ROC")
plt.legend()
plt.grid(True)
plt.show()

# ----------------------------------------------------------------------
# 2-8. Surrogate SHAP 기반 DARTS Best Model 해석(XAI)
# ----------------------------------------------------------------------

import shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# 1. DARTS 예측 결과 준비
print("y_pred shape:", y_pred.shape)

# 2. TimeSeries → numpy 변환
vals_X = test_X.all_values(copy=False)    # (N_test, F, 1)
vals_X = np.squeeze(vals_X, axis=2)       # (N_test, F)
N_test, F = vals_X.shape

input_len = best_model.input_chunk_length
print(f"Input length = {input_len}, Features = {F}")

# 3. Sliding window 생성
def create_windows(arr, win):
    X = []
    for i in range(win, len(arr)):
        X.append(arr[i-win:i, :])     # (win, F)
    return np.array(X)

X_windows = create_windows(vals_X, input_len)   # (N_samples, T, F)
y_windows = y_pred[input_len:]                  # 대응되는 예측값

print("X_windows shape:", X_windows.shape)
print("y_windows shape:", y_windows.shape)

# 4. SHAP 입력을 위해 평탄화 (Samples, T*F)
X_flat = X_windows.reshape(X_windows.shape[0], -1)
N_samples, N_features_flat = X_flat.shape
print("Flattened input shape:", X_flat.shape)

# 5. Surrogate Model (RandomForest) 학습
rf = RandomForestRegressor(
    n_estimators=400,
    max_depth=None,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_flat, y_windows)
y_hat = rf.predict(X_flat)

print("\n📌 Surrogate R² (RF vs DARTS predictions):", r2_score(y_windows, y_hat))
print("👉 Surrogate가 높을수록 SHAP 설명력이 좋음 (0.9 이상 권장)")

# 6. SHAP TreeExplainer
explainer = shap.TreeExplainer(rf)

# 분석할 샘플 수 (너무 크면 느리므로 200개 권장)
n_explain = min(200, N_samples)
X_explain = X_flat[-n_explain:]

shap_values = explainer.shap_values(X_explain)
shap_values = np.array(shap_values)   # (N_explain, T*F)

print("SHAP values shape:", shap_values.shape)

# 7. Feature 이름 생성 (예: Close_lag0, High_lag23 ...)
columns = test_X.columns  # ['Open', 'High', 'Low', 'Close']
feature_names = []

for t in range(input_len):
    lag = input_len - 1 - t
    for col in columns:
        feature_names.append(f"{col}_lag{lag}")

assert len(feature_names) == N_features_flat

# 8. Global SHAP Bar Plot (상위 중요 feature)
plt.figure(figsize=(10, 6))
plt.title("SHAP Feature Importance (Surrogate for DARTS)")
shap.summary_plot(shap_values, X_explain, feature_names=feature_names, plot_type="bar")
plt.show()

# 9. SHAP Summary Plot (color scatter)
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, X_explain, feature_names=feature_names)
plt.show()

# 10. Time × Feature Heatmap (개별 샘플 해석)
idx = -1   # 마지막 샘플 설명
sv = shap_values[idx].reshape(input_len, F).T   # (F, T)

plt.figure(figsize=(12, 3))
plt.imshow(sv, aspect="auto", cmap="bwr")
plt.colorbar(label="SHAP value")
plt.yticks(range(F), columns)
plt.xlabel(f"Time step (past {input_len} hours)")
plt.title("SHAP Heatmap: Timestep × Feature Importance")
plt.show()

# 11. Global OHLC Feature Importance (전체 lag 평균)
mean_abs_shap = np.mean(np.abs(shap_values), axis=0)     # (T*F,)
mean_abs_shap_mat = mean_abs_shap.reshape(input_len, F)  # (T, F)
feat_importance = mean_abs_shap_mat.mean(axis=0)         # (F,)

plt.figure(figsize=(5,3))
plt.bar(columns, feat_importance)
plt.ylabel("Mean |SHAP|")
plt.title("Global Feature Importance (OHLC aggregated)")
plt.show()

print("\n📌 OHLC aggregated importance:")
for col, val in zip(columns, feat_importance):
    print(f"{col}: {val:.6f}")


In [ ]:
########## 머신러닝 ##########

# ----------------------------------------------------------------------
# 3. AutoGluon 머신러닝 기반 방향성 분류 파이프라인
# ----------------------------------------------------------------------

# ----------------------------------------------------------------------
# 3-1. ML Feature Engineering (engineered features → lag 변환)
# ----------------------------------------------------------------------
df_ml = df.copy()

drop_cols = ["Date", "Open", "High", "Low", "Close"]
drop_cols = [c for c in drop_cols if c in df_ml.columns]

feature_cols = [c for c in df_ml.columns if c not in (drop_cols + ["target"])]

data_ml = df_ml[feature_cols + ["target"]].copy()

print("사용 feature 수:", len(feature_cols))
print("feature_cols:", feature_cols)

# ----------------------------------------------------------------------
# 3-2. 시계열 Split (Train/Val/Test; 85%/15%)
# ----------------------------------------------------------------------

N_ml = len(data_ml)
N_ml_train_val = int(N_ml * 0.85)

train_ml = data_ml.iloc[:N_ml_train_val].reset_index(drop=True)
test_ml  = data_ml.iloc[N_ml_train_val:].reset_index(drop=True)

print("ML train size:", len(train_ml), "test size:", len(test_ml))

# ----------------------------------------------------------------------
# 3-3. AutoGluon TabularPredictor 학습
# ----------------------------------------------------------------------

label = "target"

predictor = TabularPredictor(
    label=label,
    problem_type="binary",
    eval_metric="f1"
).fit(
    train_data=train_ml,
    presets="good_quality",
    time_limit=600,    # 10분
)

leaderboard = predictor.leaderboard(silent=True)
print("\nAutoGluon Leaderboard:")
print(leaderboard)

# ----------------------------------------------------------------------
# 3-4. AutoGluon 방향성 예측 성능 평가 (Accuracy / F1 / CM / ROC)
# ----------------------------------------------------------------------

X_test_ml = test_ml[feature_cols]
y_test_ml = test_ml[label]

pred_ml      = predictor.predict(test_ml)
proba_ml_df  = predictor.predict_proba(test_ml)

# 양성 클래스(1)의 확률
if isinstance(proba_ml_df, pd.DataFrame):
    if 1 in proba_ml_df.columns:
        prob_ml = proba_ml_df[1].values
    else:
        prob_ml = proba_ml_df.iloc[:, -1].values
else:
    prob_ml = proba_ml_df

acc_ml = accuracy_score(y_test_ml, pred_ml)
f1_ml  = f1_score(y_test_ml, pred_ml)
cm_ml  = confusion_matrix(y_test_ml, pred_ml)

fpr_ml, tpr_ml, thr_ml = roc_curve(y_test_ml, prob_ml)
auc_ml = auc(fpr_ml, tpr_ml)

print("\n📌 AutoGluon ML Direction Classification")
print(f"Accuracy: {acc_ml:.4f}")
print(f"F1      : {f1_ml:.4f}")

# Confusion Matrix
plt.figure(figsize=(5, 4))
sns.heatmap(
    cm_ml,
    annot=True,
    fmt="d",
    cmap="Blues",
    linewidths=0,
    linecolor="white",
    xticklabels=["Pred Down", "Pred Up"],
    yticklabels=["True Down", "True Up"]
)
plt.grid(False)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix (AutoGluon ML)")
plt.show()

print(f"ROC AUC : {auc_ml:.4f}")
plt.figure(figsize=(5, 5))
plt.plot(fpr_ml, tpr_ml, label=f"AUC = {auc_ml:.3f}")
plt.plot([0, 1], [0, 1], linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("AutoGluon ML ROC")
plt.legend()
plt.grid(True)
plt.show()

# ----------------------------------------------------------------------
# 3-5. SHAP 기반 ML 모델 설명 (Summary / Bar / Dependence)
# ----------------------------------------------------------------------

X_train_ml = train_ml[feature_cols]
X_test_ml  = test_ml[feature_cols]

# AutoGluon 예측 함수를 SHAP용으로 래핑
def ag_predict_proba(X):
    X_df = pd.DataFrame(X, columns=feature_cols)
    proba = predictor.predict_proba(X_df)
    if isinstance(proba, pd.DataFrame):
        if 1 in proba.columns:
            return proba[1].values
        else:
            return proba.iloc[:, -1].values
    else:
        return proba

# background / explain sample 선택
background = shap.sample(X_train_ml, 200, random_state=0)
explainer  = shap.KernelExplainer(ag_predict_proba, background)

X_explain = shap.sample(X_test_ml, 200, random_state=1)

# nsamples는 속도에 따라 조절
shap_values = explainer.shap_values(X_explain, nsamples=100)

# Summary plot (feature 중요도 + 방향성)
shap.summary_plot(
    shap_values,
    X_explain,
    feature_names=feature_cols,
    show=True
)
plt.rcParams['axes.grid'] = False
# Bar plot (mean |SHAP| 기준)
shap.summary_plot(
    shap_values,
    X_explain,
    feature_names=feature_cols,
    plot_type="bar",
    show=True
)

# Dependence plot (예: 상위 important feature 하나 골라서)
top_feature = feature_cols[0]  # 실제론 summary 결과 보고 변경 권장
shap.dependence_plot(
    top_feature,
    shap_values,
    X_explain,
    feature_names=feature_cols,
    show=True
)
